# SIIM-FISABIO-RSNA COVID-19 Detection

Identify and localize COVID-19 abnormalities on chest radiographs

In [ ]:
# list input folder
! ls /kaggle/input/siim-covid19-detection -l

## Setup environment

Installing needed packages and list versions for reproducibility

In [ ]:
# ! pip install -qU "numpy>=1.20" --no-binary numpy --no-build-isolation
! pip install -q python-gdcm
# ! pip install -q pylibjpeg-libjpeg pylibjpeg-openjpeg
# ! pip install -qU "pylibjpeg==1.2" --no-binary :all:
! pip install -qU pydicom opencv-python-headless pycocotools # "torchvision==0.8" "torch==1.7"
# ! pip install -q https://github.com/PyTorchLightning/lightning-flash/archive/master.zip
! pip list | grep torch
! pip list | grep lightning
! pip list | grep dicom
! pip list | grep jpeg
! nvidia-smi

%load_ext autoreload
%autoreload 2

import pydicom  # , pylibjpeg, openjpeg, libjpeg
print(getattr(pydicom.config, "gdcm_handler").is_available())
print(getattr(pydicom.config, "pylibjpeg_handler").is_available())

## Data exploration

Checking what data do we have available and what is the labels distribution...

We start with:
- naive loading tables
- see distributions
- visualuze images

### Overview & Annotations

Starting with checking what is the provided tables...

In [ ]:
%matplotlib inline

import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

BASE_PATH = '/kaggle/input/siim-covid19-detection'
LABELS = ("Negative for Pneumonia", "Typical Appearance", "Indeterminate Appearance", "Atypical Appearance")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

**train_image_level.csv** - the train image-level metadata, with one row for each image, including both correct labels and any bounding boxes in a dictionary format. Some images in both test and train have multiple bounding boxes.

- `id` - unique image identifier
- `boxes` - bounding boxes in easily-readable dictionary format
- `label` - the correct prediction label for the provided bounding boxes

Enrich table with some countings and parsing the base name

In [ ]:
path_csv_image = os.path.join(BASE_PATH, 'train_image_level.csv')
train_images = pd.read_csv(path_csv_image, index_col="id").sort_values("StudyInstanceUID")
train_images["name"] = [v.split('_')[0] for v in train_images.index]
train_images["boxes"] = train_images["boxes"].apply(lambda v: eval(v) if not isinstance(v, float) else None)
train_images["#boxes"] = train_images["boxes"].apply(lambda v: len(v) if v else 0)

imgs_paths = [glob.glob(os.path.join(BASE_PATH, 'train', row['StudyInstanceUID'], '*', f"{row['name']}.*")) for _, row in train_images.iterrows()]
print(f"max images shall be one and is: {max([len(p) for p in imgs_paths])}")
train_images["path"] = [os.path.sep.join(p[0].split(os.path.sep)[-4:]) for p in imgs_paths]

display(train_images.head())
print(len(train_images))

In [ ]:
train_images["label_"] = train_images["label"].apply(lambda lb: lb.split()[::6])
train_images["label__"] = train_images["label_"].apply(lambda lb: set(lb))
ax = train_images["label__"].value_counts().plot.pie(ylabel="", autopct="%.1f%%")

In [ ]:
labels_none = [lb for lb in train_images["label"] if lb.startswith("none")]
print(set(labels_none))

In [ ]:
labels_1 = [tuple(lb.split()[1::6]) for lb in train_images["label"]]
print(set(labels_1))

**train_study_level.csv** - the train study-level metadata, with one row for each study, including correct labels.

- `id` - unique study identifier
- `Negative for Pneumonia` - 1 if the study is negative for pneumonia, 0 otherwise
- `Typical Appearance` - 1 if the study has this appearance, 0 otherwise
- `Indeterminate Appearance`  - 1 if the study has this appearance, 0 otherwise
- `Atypical Appearance` - if the study has this appearance, 0 otherwise


In [ ]:
path_csv_study = os.path.join(BASE_PATH, 'train_study_level.csv')
train_study = pd.read_csv(path_csv_study, index_col="id").sort_values("id")
train_study["id_"] = [v.split('_')[0] for v in train_study.index]
train_study["class"] = [np.argmax(row.values) for _, row in train_study[list(LABELS)].iterrows()]
display(train_study.head())
print(len(train_study))

From previous we ca see nb images is larger nb studies...

In [ ]:
counts = train_images["StudyInstanceUID"].value_counts()
display(dict(enumerate(np.bincount(counts))))
ax = counts.hist(bins=2*max(counts))

See sanity chek that sumof labels is equls to nb samples and show case/label distibution

In [ ]:
train_study_ids = set(train_study["id_"])
miss = [id_ for id_ in train_images["StudyInstanceUID"] if id_ not in train_study_ids]
print(f"Missed: {len(miss)}")
print(f"{len(train_study)} == {train_study[list(LABELS)].sum().sum()}")

ax = train_study[list(LABELS)].sum().plot.pie(ylabel="", autopct="%.1f%%")

### Fuse the two tables

lets trasfer the labels to the images

In [ ]:
train_images = pd.merge(train_images, train_study, how="left", left_on="StudyInstanceUID", right_on="id_")
display(train_images.head())

In [ ]:
fig, axarr = plt.subplots(ncols=2, figsize=(7, 3))

train_images_none = train_images[train_images["label"].str.startswith("none")]
axarr[0].set_title("classes with labels None")
ax = train_images_none["class"].value_counts().plot.pie(ax=axarr[0], ylabel="", autopct="%.1f%%")

train_images_other = train_images[~ train_images["label"].str.startswith("none")]
axarr[1].set_title("classes with any labels")
ax = train_images_other["class"].value_counts().plot.pie(ax=axarr[1], ylabel="", autopct="%.1f%%")

### Show sample image

loading the mages from DICOM format and show then in standard figures...

In [ ]:
import pydicom
from pydicom.pixel_data_handlers import apply_voi_lut

idx_ = 0
dicom_path = os.path.join(BASE_PATH, train_images["path"][idx_])
dicom = pydicom.dcmread(dicom_path)
print(vars(dicom).keys())
print(dicom)
img = apply_voi_lut(dicom.pixel_array, dicom)


import matplotlib.pyplot as plt
from matplotlib import patches

fig, ax = plt.subplots()
ax_im = ax.imshow(img, cmap="gray")
for bbox in train_images["boxes"][idx_]:
    # Create a Rectangle patch
    rect = patches.Rectangle((bbox['x'], bbox['y']), bbox['width'], bbox['height'], linewidth=1, edgecolor='r', facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)

_= plt.colorbar(ax_im)

### Samples per class

group images per class and show a few sample images per class together with detection bounding boxes

In [ ]:
import cv2
from copy import deepcopy

def load_image(path_file: str, meta: dict, spacing: float = 1.0, percentile: bool = True):
    dicom = pydicom.dcmread(path_file)
    try:
        img = apply_voi_lut(dicom.pixel_array, dicom)
    except RuntimeError as err:
        print(err)
        return None, {}
    if dicom.PhotometricInterpretation == 'MONOCHROME1':
        img = img.max() - img
    p_low = np.percentile(img, 1) if percentile else img.min()
    p_high = np.percentile(img, 99) if percentile else img.max()
    # normalize
    img = (img.astype(float) - p_low) / (p_high - p_low)
    meta.update(dict(
        boxes=deepcopy(meta.get("boxes")) or [],
        body=dicom.BodyPartExamined,
        interpret=dicom.PhotometricInterpretation,
        spacing=dicom.ImagerPixelSpacing,
    ))
    if spacing:
        factor = np.array(meta['spacing']) / spacing
        dims = tuple((np.array(img.shape[::-1]) * factor).astype(int))
        img = cv2.resize(img, dsize=dims, interpolation=cv2.INTER_LINEAR)
        for bbox in meta["boxes"]:
            bbox['x'] *= factor[0]
            bbox['y'] *= factor[1]
            bbox['width'] *= factor[0]
            bbox['height'] *= factor[1]
        meta.update(dict(spacing=(spacing, spacing)))
    return img, meta

In [ ]:
NB_SAMPLES = 5
NB_CLASSES = max(train_images["class"]) + 1
rect_property = dict(linewidth=1, edgecolor='r', facecolor='none')

fig, axarr = plt.subplots(nrows=NB_CLASSES, ncols=NB_SAMPLES, figsize=(NB_SAMPLES * 4, NB_CLASSES * 5))
for cls, df in train_images.groupby("class"):
    for i, (_, row) in enumerate(df[:NB_SAMPLES].iterrows()):
        img, meta = load_image(os.path.join(BASE_PATH, row["path"]), dict(row), spacing=1.)
        axarr[cls, i].set_title(f"label: {cls}; body: {meta['body']}\n interpret: {meta['interpret']}\n spacing: {meta['spacing']}")
        if img is None:
            continue
        _ = axarr[cls, i].imshow(img.astype(float) / img.max(), cmap="gray")
        if not meta["boxes"]:
            continue
        for bbox in meta["boxes"]:
            rect = patches.Rectangle((bbox['x'], bbox['y']), bbox['width'], bbox['height'], **rect_property)
            axarr[cls, i].add_patch(rect)

### Number of detections per case

Histgram of number of detections in subject depending on type

In [ ]:
counts, cases = [], []
for cls, df in train_images.groupby("class"):
    counts.append(dict(df["#boxes"].value_counts()))
    cases.append(LABELS[cls])

df = pd.DataFrame(counts, index=cases)
display(df)
ax = df[sorted(df.columns)].T.plot.bar(grid=True, xlabel="#boxes per image", ylabel="#images per class", figsize=(7, 3))

In [ ]:
# from tqdm.autonotebook import tqdm

# train_images['BodyPart'] = [pydicom.dcmread(os.path.join(BASE_PATH, p)).BodyPartExamined for p in tqdm(train_images['path'])]

# counts, cases = [], []
# for cls, df in train_images.groupby("class"):
#     counts.append(dict(df["BodyPart"].value_counts()))
#     cases.append(LABELS[cls])

# ax = pd.DataFrame(counts, index=cases).T.plot.bar(grid=True, xlabel="BodyPart", ylabel="#images", figsize=(8, 3))

## Convert to COCO

The *.txt file specifications are:

- One row per object - class x_center y_center width height
- Box coordinates must be in normalized xywh format (from 0 - 1)
- Class numbers are zero-indexed

<img src="https://user-images.githubusercontent.com/26833433/91506361-c7965000-e886-11ea-8291-c72b98c25eec.jpg" width="480">

In [ ]:
def convert_boxes_to_coco(meta, image_hw):
    # ih, iw = img.shape[:2]
    ih, iw = image_hw
    bboxes = []
    for bbox in meta["boxes"]:
        # cls, x_center, y_center, width, height
        bboxes.append({
            "cls": meta["class"],
            "x_center": float(bbox['x'] + bbox['width'] / 2) / iw,
            "y_center": float(bbox['y'] + bbox['height'] / 2) / ih,
            "width":  bbox['width'] / iw,
            "height": bbox['height'] / ih
        })
    return bboxes

prepare the new dataset folders

In [ ]:
PATH_OUT = "/kaggle/working"
PATH_OUT_IMAGE = "/kaggle/working/images"
PATH_OUT_LABEL = "/kaggle/working/labels"
SPACING = 1.0

for d in (PATH_OUT_IMAGE, PATH_OUT_LABEL):
    os.makedirs(d, exist_ok=True)
    for dd in ("train", "test"):
        os.makedirs(os.path.join(d, dd), exist_ok=True)

conver train images and save metadata to the overview table

In [ ]:
from tqdm.autonotebook import tqdm
from multiprocessing import Pool
from functools import partial


def conver_image(id_row, dir_name):
    _, row = id_row
    # phase = "train" if np.random.random() < 0.8 else "valid"
    img, meta = load_image(os.path.join(BASE_PATH, row['path']), dict(row), spacing=SPACING)
    plt.imsave(os.path.join(PATH_OUT_IMAGE, dir_name, f"{row['name']}.jpg"), img, cmap='gray')
    bboxes = convert_boxes_to_coco(meta, image_hw=img.shape[:2])
    df = pd.DataFrame(bboxes)[["cls", "x_center", "y_center", "width", "height"]] if bboxes else pd.DataFrame(bboxes)
    df.to_csv(os.path.join(PATH_OUT_LABEL, dir_name, f"{row['name']}.txt"), sep=" ", index=None, header=None)
    meta.update({"bboxes": bboxes, "image_size": img.shape})
    return meta

metas = []
pool = Pool(os.cpu_count())
for meta in pool.map(partial(conver_image, dir_name="train"), tqdm(train_images.iterrows(), total=len(train_images))):
    metas.append(meta)
pool.close()
pool.join()

Creating the COCO coordinate file which contains:

- bounding boxes
- images with dimensions
- class describtion

In [ ]:
import json

annots = []
running_id = 0
for idx, meta in enumerate(metas):
    ih, iw = meta["image_size"]
    for i, box in enumerate(meta["bboxes"]):
        w = int(box['width'] * iw)
        h = int(box['height'] * ih)
        x = int(box['x_center'] * iw) - np.ceil(w / 2.)
        y = int(box['y_center'] * ih) - np.ceil(h / 2.)
        rec = {
            "id": running_id,
            "image_id": idx,
            "category_id": meta['class'],
            "area": w * h,
            "bbox": [max(0, x), max(0, y), w, h],
            "segmentation": [],
            "iscrowd": 0,
        }
        annots.append(rec)
        running_id += 1

coco = {
    "annotations": annots,
    "categories": [{"id": i, "name": n, "supercategory": ""} for i, n in enumerate(LABELS)],
    "images": [{"id": idx, "file_name": f"{meta['name']}.jpg", "height": meta["image_size"][0], "width": meta["image_size"][1]} for idx, meta in enumerate(metas)],
}

path_json = os.path.join(PATH_OUT, "covid_train.json")
with open(path_json, "w") as fp:
    json.dump(coco, fp)

Converting the test images

In [ ]:
found_images = glob.glob(os.path.join(BASE_PATH, 'test', '*', '*', '*.dcm'))

test_images = pd.DataFrame({
    "name": os.path.splitext(os.path.basename(p))[0],
    "path": os.path.sep.join(p.split(os.path.sep)[-4:])
} for p in found_images)
display(test_images.head())

pool = Pool(os.cpu_count())
list(pool.imap_unordered(partial(conver_image, dir_name="test"), tqdm(test_images.iterrows(), total=len(test_images))))
pool.close()
pool.join()

In [ ]:
! cd /kaggle/working
! zip covid-dataset.zip -q -r *

## Training with Flash

In [ ]:
! rm -rf lightning-flash
! pip uninstall -y lightning-flash
! git clone https://github.com/PyTorchLightning/lightning-flash.git
! cd lightning-flash && git checkout feature/icevision && pip install -q .[image]
# ! pip install -q https://github.com/PyTorchLightning/lightning-flash/archive/refs/heads/feature/icevision.zip#egg=lightning-flash[image]
! pip uninstall -y fiftyone wandb
# ! pip install -q effdet

In [ ]:
import flash
from flash.image import ObjectDetectionData, ObjectDetector

# 1. Create the DataModule
dm = ObjectDetectionData.from_coco(
    train_folder=os.path.join(PATH_OUT_IMAGE, 'train'),
    train_ann_file=os.path.join(PATH_OUT, "covid_train.json"),
    val_split=0.1,
    batch_size=6,
    image_size=640,
)

In [ ]:
# 2. Build the task
model = ObjectDetector(
    head="efficientdet",
    backbone="tf_d3_ap",
    learning_rate=1.5e-5,
    num_classes=dm.num_classes,
    image_size=640,
)

### Run traning

In [ ]:
import pytorch_lightning as pl
print(pl.__version__)
logger = pl.loggers.CSVLogger(save_dir='logs/')
                              
# 3. Create the trainer and finetune the model
trainer = flash.Trainer(
    max_epochs=20,
    gpus=1,
    precision=16,
    accumulate_grad_batches=12,
    logger=logger,
    val_check_interval=0.5,
)
trainer.finetune(model, datamodule=dm, strategy="freeze_unfreeze")

# 3. Save the model!
trainer.save_checkpoint("object_detection_model.pt")

### Show training charts

In [ ]:
metrics = pd.read_csv(f'{trainer.logger.log_dir}/metrics.csv')
display(metrics.head())

aggreg_metrics = []
agg_col = "epoch"
for i, dfg in metrics.groupby(agg_col):
    agg = dict(dfg.mean())
    agg[agg_col] = i
    aggreg_metrics.append(agg)

df_metrics = pd.DataFrame(aggreg_metrics)
df_metrics[['loss', 'class_loss', 'box_loss']].plot(grid=True, legend=True, xlabel=agg_col)
df_metrics[['Precision (IoU=0.50:0.95,area=all)', 'Recall (IoU=0.50:0.95,area=all,maxDets=100)']].plot(grid=True, legend=True, xlabel=agg_col)

### Predict labels for test images

In [ ]:
# 4. Detect objects in a few images!
predictions = []
model.to("cuda")
for _, row in tqdm(test_images.iterrows(), total=len(test_images)):
    p_img = os.path.join(PATH_OUT_IMAGE, "test", f"{row['name']}.jpg")
    preds = model.predict([p_img])
    rec = {**dict(row), "predictions": preds[0]}
    predictions.append(rec)

In [ ]:
print(predictions[0])
display(predictions[0]['predictions'].as_dict)

In [ ]:
pred_boxes = [len(p['predictions'].as_dict()['detection']['bboxes']) for p in predictions]
print(dict(enumerate(np.bincount(pred_boxes))))
print(pred_boxes)

In [ ]:
# ! ls /kaggle/working/images/test/
# ! ls /kaggle/working/images/test/